In [17]:
#necessary imports
import sqlalchemy as sqlconn
import pandas as pd
import numpy as np
from SQLConfig import config
import pickle


from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
#PostgresSQL configuration
param=config()
url = f"postgresql+psycopg2://{param['user']}:{param['password']}@{param['host']}:{param['port']}/{param['database']}"
engine = sqlconn.create_engine(url)

In [3]:
def get_query(query:str):
    """Executes and SQL query and returns the results in dataframe format 

    Args:
        query (string): The SQL query to execute

    Returns:
        pandas dataframe: result of the SQL query
    """
    with engine.connect().execution_options(autocommit=True) as conn:
        df=pd.read_sql(sqlconn.text(query),conn)
        return df

In [4]:
def set_query(query:str):
    """Used to execute SQL queries that does not return any result such as INSERT,
       DELETE, CREATE commands

    Args:
        query (string): THE SQL query to execute
    """
    with engine.connect().execution_options(autocommit=True) as conn:
        conn.execute(sqlconn.text(query))

In [ ]:
def build_test_set(build_sql=False,pandas_data=None):
    """builds a testing set with two options
        that is to build from SQL database if the data exists in the raw form
        else provide a pandas dataframe in the right format.

    Args:
        build_sql (bool, optional): if true then build using SQL. Defaults to False.
        pandas_data (None, optional): if true then build using input dataframe. Defaults to None.

    Raises:
        TypeError: if pandas_data is not an instance of pandas.DataFrame

    Returns:
        X: input features
        y: output labels
    """
    if build_sql:
        sql="""
            WITH customer_data AS (
                SELECT r.customer_id
                        , r.receipt_id
                        , purchased_at
                        , quantity
                        , value 
                FROM receipts r
                LEFT JOIN
                (   
                    SELECT receipt_id, SUM(qty) quantity,SUM(value) value
                    FROM receipt_lines
                    GROUP BY 1
                ) t USING (receipt_id)
            ), 
            customer_all_data AS
                (
                    SELECT customer_id
                        , purchased_at
                        , SUM(quantity) quantity
                        , SUM(value) value
                        , COUNT(receipt_id) visits
                    FROM customer_data
                    GROUP BY 1,2
                    ORDER BY 1,2
                ),
            customer_all_dates AS
                (
                    SELECT DISTINCT customer_id
                        ,purchased_date::DATE purchased_at
                    FROM receipts
                    JOIN LATERAL generate_series('2020-07-22'::DATE,'2022-03-22'::DATE,'1 day'::INTERVAL) purchased_date ON TRUE
                ),
            customer_all_features AS
                (
                    SELECT r.customer_id
                        , (r.purchased_at::DATE - '2020-07-22'::DATE)+1 purchased_day
                        , extract(month FROM purchased_at)  month_of_year
                        , COALESCE(visits,0) visits
                        , COALESCE(quantity,0) quantity
                        , COALESCE(value,0) value
                    FROM customer_all_dates r
                    LEFT JOIN customer_all_data s USING (customer_id,purchased_at)
                    ORDER BY 1,2
                ),
            WITH ml_data_engineered AS (
                SELECT *
                FROM
                (
                    SELECT customer_id
                        , purchased_day ref_day
                        , month_of_year
                        , MAX(CASE WHEN value>0 THEN purchased_day ELSE 0 END) OVER (PARTITION BY customer_id ORDER BY purchased_day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) recent_purchased_day
                        , SUM(visits) OVER (PARTITION BY customer_id ORDER BY purchased_day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_visits
                        , MIN(CASE WHEN value>0 THEN purchased_day ELSE 'infinity'::FLOAT END) OVER (PARTITION BY customer_id ORDER BY purchased_day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) min_purchased_day
                        
                        , SUM(value) OVER (PARTITION BY customer_id ORDER BY purchased_day ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS purchase_since_2m
                        , (CASE WHEN (SUM(value) OVER (PARTITION BY customer_id ORDER BY purchased_day ROWS BETWEEN {churn_def-8} PRECEDING AND 7 FOLLOWING))=0 THEN 1 ELSE 0 END) churn
                    FROM customer_all_features
                ) r
                WHERE ref_day>=60+85
            ),
            ml_data_churn AS (
                SELECT customer_id
                    , ref_day
                    , month_of_year
                    , recent_purchased_day
                    , total_visits
                    , (CASE WHEN min_purchased_day>0 THEN (CASE WHEN (ref_day-min_purchased_day)='-infinity'::FLOAT THEN 0 ELSE (ref_day-min_purchased_day)+1 END) ELSE 0 END) customer_age
                    , purchase_since_2m
                    , churn
                FROM ml_data_engineered
                ORDER BY 1,2
            )
        """
        Xy=get_query(sql)
        X=Xy.drop(columns=['churn'])
        y=Xy.churn
        return X,y
    if isinstance(pandas_data,pd.DataFrame):
        X=pandas_data.drop(columns=['churn'])
        y=pandas_data.churn
        return X,y
    raise TypeError("pandas_data must be a pandas DataFrame!")
    

In [ ]:
X_test,y_test=build_test_set() #input pandas data or build SQL

In [ ]:
#load pre-trained model
model=pickle.load(open('churn_model.pkl','rb'))

In [ ]:
#predict the labels of test set
y_pred=model.predict(X_test)

In [ ]:
#scorer arguments
kwargs=dict(
    accuracy=True,
    precision=True,
    recall=True,
    f1_score=False,
    area_roc=False,
    balanced_accuracy=False
)

def scores(**kwargs):
    """prints the accuracy, precision and recall scores by default.
    Other scores available such as F1 score, Area under ROC curve and balanced accuracy.
    """
    if kwargs.get('accuracy'):
        print("Accuracy score:",np.round(accuracy_score(y_test,y_pred)))
    if kwargs.get('precision'):
        print("Precision score:",np.round(precision_score(y_test,y_pred)))
    if kwargs.get('recall'):
        print("Recall score:",np.round(recall_score(y_test,y_pred)))
    if kwargs.get('f1_score'):
        print("F1 score:",np.round(f1_score(y_test,y_pred)))
    if kwargs.get('balanced_accuracy'):
        print("Balanced Accuracy score:",np.round(balanced_accuracy_score(y_test,y_pred)))
    if kwargs.get('area_roc'):
        y_pred_prob=model.predict_proba(X_test)
        print("Area under ROC score:",np.round(roc_auc_score(y_test,y_pred_prob)))
    

In [ ]:
#Display scores
scores()